In [4]:
# import internal .py modules
import file_path_management as fpath
import public_library as plib

In [5]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [6]:
# www.frontiersin.org
def www_frontiersin_org(url):
    os.environ['WDM_LOG'] = '0'
    options = Options()
    options.add_argument('--headless')
    driver = webdriver.Chrome()
    driver.get(url)
    # load the webpage
    error_label = 0
    while(error_label == 0):
        try:
            driver = webdriver.Chrome()
            driver.get(url)
            # WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//p[text()='Consent']"))).click()
            error_label = 1
        except:
            print("Extracting content from:" + url + " failed, retrying... This might take longer than 5 minutes...")
            time.sleep(5*60)
            error_label = 0
    
    # extract information from loaded webpage
    try:
        doi = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[1]/div[1]/div[1]/div/div[2]/span[2]/a").text
    except:
        doi = np.nan
    try:
        pmid = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[1]/div[1]/div[2]/div[2]/a").text
    except:
        pmid = np.nan
    try:
        pmcid = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[1]/div[1]/div[2]/div[1]/span[2]").text
    except:
        pmcid = np.nan
    try:
        title = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[1]/h1").text
    except:
        title = np.nan
    try:
        abstract = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[4]/div[2]/p").text
    except:
        abstract = np.nan
    try:
        keywords = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[4]/div[3]/span").text
    except:
        keywords = np.nan
    try:
        intro = driver.find_element(By.XPATH, '//*[@id="S1"]').text
    except:
        intro = np.nan
    try:
        pdf_link = driver.find_element(By.XPATH, "/html/body/main/article/section[3]/div/div[1]/div[1]/div[1]/div[1]/div/div[2]/span[2]/a").get_attribute('href')
    except:
        pdf_link = np.nan

    driver.quit()

    info = {
        "doi": doi,
        "pmid": pmid,
        "pmcid": pmcid,
        "title": title,
        "abstract": abstract,
        "keywords": keywords,
        "introduction": intro,
        "pdf_link": pdf_link
    }

    return info
# --------------------start of test code--------------------
url = "https://www.frontiersin.org/articles/10.3389/fncir.2015.00079/full"
info = www_frontiersin_org(url)
print(info["doi"])
print(info["pmid"])
print(info["pmcid"])
print(info["abstract"])
print(info["keywords"])
print(info["introduction"])
print(info["pdf_link"])
# ---------------------end of test code---------------------